In [1]:
import requests
import pandas as pd
import openpyxl
import re


https://datainfogreffe.fr/dashboard/documentation

In [2]:
arx_book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\Projet Srapping_API - Partie 2.xlsx')


In [25]:
sheet = arx_book.active

rows = sheet.max_row 
#la fonction ne renvoie pas la dernière ligne non-vide mais toutes les lignes d'où la condition !=None en bas
cols = sheet.max_column

SIRET = []
noms = []
pays = []

for row in sheet.iter_rows(min_row=2, min_col=2, max_row=rows, max_col=2) :
    for cell in row:
        if cell.value != None :
            noms.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=3, max_row=rows, max_col=3) :
    for cell in row:
        if cell.value != None : 
            SIRET.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=4, max_row=rows, max_col=4) :
    for cell in row:
        if cell.value != None : 
            pays.append(cell.value)
    
data = pd.DataFrame({'SIRET': SIRET, 'pays': pays, },
                      index = noms)

data

,SIRET,pays
AG2R La Mondiale,502 858 418 00028,France
Astera,570 501 551 00016,France
Groupe PRO BTP,394 164 966 00019,France
Crédit Mutuel,588 505 354 00013,France
Système U,304 602 956 00142,France
...,...,...
Greenergy,2809935,Royaume-Uni
Ineos,3851680,Royaume-Uni
Wittington Investments,366054,Royaume-Uni
Betfred Group,7717019,Royaume-Uni


# Fiche identite (test)

Il est possible qu'il faille changer le token

In [4]:
SIRET_test = ['50285841800028','35600000000000']
noms_test = ['AG2R La Mondiale']
prefixe = 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/'
suffixe = '?token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'

def get_pages(U):
    pages = []
    for i in range(0,len(U)):
        SIREN = U[i]
        j = prefixe + SIREN[:-5] + suffixe #enlever les 5 derniers pour avoir le SIREN à partir du SIRET
        pages.append(j)
    return pages

pages_test = get_pages(SIRET_test)
pages_test

['https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/502858418?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/356000000?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ']

In [14]:
response = requests.get(pages_test[0])
print(response.text)
print(response.encoding)
print(response.url)
# print(response.content)

{"Metadata":{"CreditsUsed":1,"CreditsLeft":192},"Data":{"Siren":"356000000","Denomination":"La Poste","CodeGreffe":"7501","LibelleGreffe":"PARIS","NomCommercial":null,"Enseigne":null,"Sigle":"","Adresse":{"Residence":"","Localite":null,"NumeroVoie":"9","TypeVoie":null,"NomVoie":"RUE DU COLONEL PIERRE AVIA","AdresseConcat":"9 RUE DU COLONEL PIERRE AVIA","CodePostal":"75015","BureauDistributeur":"PARIS","CodePays":null},"LibelleFormeJuridique":"Société anonyme","CodeActivite":"5310Z","LibelleActivite":"Activités de poste dans le cadre d'une obligation de service universel","DateImmatriculation":"1992-03-19","EtatRadiation":"false","DateRadiation":null}}
utf-8
https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/356000000?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ


In [7]:
dict = response.json()
dict

{'Metadata': {'CreditsUsed': 1, 'CreditsLeft': 192},
 'Data': {'Siren': '356000000',
  'Denomination': 'La Poste',
  'CodeGreffe': '7501',
  'LibelleGreffe': 'PARIS',
  'NomCommercial': None,
  'Enseigne': None,
  'Sigle': '',
  'Adresse': {'Residence': '',
   'Localite': None,
   'NumeroVoie': '9',
   'TypeVoie': None,
   'NomVoie': 'RUE DU COLONEL PIERRE AVIA',
   'AdresseConcat': '9 RUE DU COLONEL PIERRE AVIA',
   'CodePostal': '75015',
   'BureauDistributeur': 'PARIS',
   'CodePays': None},
  'LibelleFormeJuridique': 'Société anonyme',
  'CodeActivite': '5310Z',
  'LibelleActivite': "Activités de poste dans le cadre d'une obligation de service universel",
  'DateImmatriculation': '1992-03-19',
  'EtatRadiation': 'false',
  'DateRadiation': None}}

La réponse est un dictionnaire dont on peut extraire les informations.

# Comptes Annuels (test)

Il faut choisir l'année

In [65]:
prefixe2 = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
id = '356000000'
annee = '2019'
suffixe2 = '&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'
url2 = prefixe2 + id + '?millesime=' + annee + suffixe2
url2

'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/356000000?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'

In [66]:
response3 = requests.get(url2)

La réponse fournie les comptes sous forme de lettres, voir la documentation pour savoir à quoi cela correspond.

In [67]:
dict3 = response3.json()
dict3

{'Metadata': {'CreditsUsed': 5, 'CreditsLeft': 195},
 'Data': {'CodeGreffe': '7501',
  'Siren': '356000000',
  'Denomination': 'LA POSTE',
  'Millesime': '2019',
  'TypeComptes': 'sociaux',
  'Liasse': '2050',
  'Devise': 'EUR',
  'DateCloture': '2019-12-31',
  'DureeExo': '12',
  'DureeExoPrecedent': '12',
  'Postes': [{'CodeInfogreffe': 'AAJ', 'Valeur': '1470000000'},
   {'CodeInfogreffe': 'AAK', 'Valeur': '1110000000'},
   {'CodeInfogreffe': 'A1N', 'Valeur': '360000000'},
   {'CodeInfogreffe': 'AAN', 'Valeur': '282000000'},
   {'CodeInfogreffe': 'AAO', 'Valeur': '161000000'},
   {'CodeInfogreffe': 'A1R', 'Valeur': '121000000'},
   {'CodeInfogreffe': 'AAT', 'Valeur': '3662000000'},
   {'CodeInfogreffe': 'AAU', 'Valeur': '2938000000'},
   {'CodeInfogreffe': 'A1U', 'Valeur': '724000000'},
   {'CodeInfogreffe': 'ABH', 'Valeur': '12625000000'},
   {'CodeInfogreffe': 'ABI', 'Valeur': '94000000'},
   {'CodeInfogreffe': 'A2B', 'Valeur': '12531000000'},
   {'CodeInfogreffe': 'ABJ', 'Valeur':

In [21]:
dict3['Data']['Postes'][0]['CodeInfogreffe']

'AAJ'

Dans la partie Data, on prend le dictionnaire numéro 0 de la sous-partie Poste. Dans ce distionnaire, on prend le "CodeInfogreffe"

# Problèmes rencontrées : données non disponibles

**Aucun crédit n'est utilisé quand on a un erreur**

<h3>{'Code': 404, 'Message': 'Bilan non trouvé'}</h3>

In [1]:
NoFinanceDataList = [   #SireneSansFinanceData
    "502858418",
    "394164966",
    "588505354",
    "450527916",
    "775577018",
    "417591971",
    "882245293",
    "752610147",
    "782092290",
    "781452511",
    "775663438",
    "332789296",
    "841645690",
    "348939513",
    "479878340"
]

In [21]:
def url_entreprise_problematique(NoFinanceDataList, annee1 ):#, annee2):
    liste_url = []
    for x in NoFinanceDataList:
        url = ''
        prefixe = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
        id = x
        annee = annee1
        suffixe = '&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'
        url = prefixe + id + '?millesime=' + annee + suffixe
        liste_url.append(url)
    return liste_url
                                 
liste_url_2019 = url_entreprise_problematique(NoFinanceDataList, '2019') #seulement en 2019 pour l'instant 
liste_url_2019 

['https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/502858418?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/394164966?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/588505354?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/450527916?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/775577018?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/417591971?millesime=2019&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/E

In [22]:
def entreprise_problematique(liste_url, NoFinanceDataList):
    for i in range(len(liste_url)):
        response = requests.get(liste_url[i])
        print("Résultat pour l'entreprise : " + NoFinanceDataList[i])
        print(response.json())
        print("-----------------------------------------------------------------------------------------------------------")

In [23]:
entreprise_problematique(liste_url_2019, NoFinanceDataList)

Résultat pour l'entreprise : 502858418
{'Code': 404, 'Message': 'Bilan non trouvé'}
-----------------------------------------------------------------------------------------------------------
Résultat pour l'entreprise : 394164966
{'Code': 404, 'Message': 'Bilan non trouvé'}
-----------------------------------------------------------------------------------------------------------
Résultat pour l'entreprise : 588505354
{'Code': 404, 'Message': 'Bilan non trouvé'}
-----------------------------------------------------------------------------------------------------------
Résultat pour l'entreprise : 450527916
{'Code': 404, 'Message': 'Bilan non trouvé'}
-----------------------------------------------------------------------------------------------------------
Résultat pour l'entreprise : 775577018
{'Code': 404, 'Message': 'Bilan non trouvé'}
-----------------------------------------------------------------------------------------------------------
Résultat pour l'entreprise : 417591971
{

<h3>{'Code': 500, 'Message': 'The request timed out'}</h3>

In [51]:
prefixe2 = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
id = '570501551'
annee = '2019'
suffixe2 = '&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ'
url22 = prefixe2 + id + '?millesime=' + annee + suffixe2
response33 = requests.get(url22)
response33

<Response [500]>

In [62]:
response33.status_code

500

In [2]:
prefixe2 = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
id = '304602956'
annee = '2019'
suffixe2 = '&token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ'
url22 = prefixe2 + id + '?millesime=' + annee + suffixe2
response44 = requests.get(url22)
response44

<Response [500]>

In [5]:
response44.text

'<html><head><title>500 - The request timed out.</title></head><body>  <font color ="#aa0000">         <h2>500 - The request timed out.</h2></font>  The web server failed to respond within the specified time.</body></html>'

<h3>{'Code': 402, 'Message': 'Crédits insuffisants.'}</h3>

In [64]:
prefixe2 = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
id = '356000000'
annee = '2019'
suffixe2 = '&token=TlggsZq9cxMULcq9RmFzxf7lyhzZjQGOeK03VQdeed7r1Pl29iNkKhAWrSagoWV'
url2 = prefixe2 + id + '?millesime=' + annee + suffixe2
response4 = requests.get(url2)
dict4 = response3.json()
dict4

{'Code': 402, 'Message': 'Crédits insuffisants.'}

**Une lien :**

https://opendata.datainfogreffe.fr/explore/embed/dataset/chiffres-cles-2019/table/?sort=ca_1&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6ImNoaWZmcmVzLWNsZXMtMjAxOSIsIm9wdGlvbnMiOnsic29ydCI6ImNhXzEifX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImxpbmUiLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJjb2RlX3Bvc3RhbCIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiM2NmMyYTUifV0sInhBeGlzIjoiZGF0ZV9kZV9jbG90dXJlX2V4ZXJjaWNlXzMiLCJtYXhwb2ludHMiOiIiLCJ0aW1lc2NhbGUiOiJ5ZWFyIiwic29ydCI6IiJ9XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D

# Exploitation : Construction des Dataframes et du document Excel final

**Récupération des numéros SIRET depuis le fichier ARX :**

In [46]:
# arx_book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\Projet Srapping_API - Partie 2.xlsx')
arx_book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\Projet Srapping_API - test.xlsx')


sheet = arx_book.active

rows = sheet.max_row 
#la fonction ne renvoie pas la dernière ligne non-vide mais toutes les lignes d'où la condition !=None en bas
cols = sheet.max_column

SIRET = []
noms = []
pays = []

for row in sheet.iter_rows(min_row=2, min_col=2, max_row=rows, max_col=2) :
    for cell in row:
        if cell.value != None :
            noms.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=3, max_row=rows, max_col=3) :
    for cell in row:
        if cell.value != None : 
            SIRET.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=4, max_row=rows, max_col=4) :
    for cell in row:
        if cell.value != None : 
            pays.append(cell.value)
    
data = pd.DataFrame({'SIRET': SIRET, 'pays': pays, },
                      index = noms)
data

,SIRET,pays
AG2R La Mondiale,502 858 418 00028,France
Astera,570 501 551 00016,France
Groupe PRO BTP,394 164 966 00019,France
Crédit Mutuel,588 505 354 00013,France
Système U,304 602 956 00142,France
Cooperl Arc Atlantique,383 986 874 00014,France
Covéa,450 527 916 00024,France
Arkéa,775 577 018 00499,France
Cristal Union,421 343 369 00011,France
GCC Construction,407 794 551 00231,France


**Creation des tables vides :**

In [47]:
CodeGreffe = []
Names = []
Sirenes = []
Adresses = []
ZipCodes = []
Villes = []
Pays = []
Secteurs = []
Descriptions_FR = []

# theBook = xlrd.open_workbook("Data_File.xlsx")
# theSheet = theBook.sheet_by_name("Sociétés non cotées")

**Construction des urls pour appeler l'API et récupérer les informations entreprises :**

In [48]:
def url_info_entreprise(data): #attention à mettre l'année en format String
    liste_url = []
    for i in range(0,len(data)):
        url = ''
        if data['pays'][i] == 'France':
            SIRET = data['SIRET'][i]
            SIRET = re.sub(' ',  '',SIRET)
            SIRET = re.sub('\u200e',  '',SIRET) #une des lignes est problématique
            SIRET = SIRET[:-5] #traitement pour avoir le SIREN
            prefixe = 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/'
            suffixe = '?token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'
            j = prefixe + SIRET + suffixe
            liste_url.append(j)
    return liste_url

        
liste_url_info = url_info_entreprise(data)
print("Coût en Credit Datainfo : " + str(len(liste_url_info)))
display(liste_url_info)

Coût en Credit Datainfo : 50


['https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/502858418?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/570501551?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/394164966?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/588505354?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/304602956?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/383986874?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2qsdSIJ',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/FicheIdentite/450527916?token=hO7YpAmluPh44Ar5zv2Ktu8mub8aNrXbmTIlwjtSd1AuNA3iSPVawF2q

**Appeler l'API et remplir les tableaux :**

In [49]:
def information_entreprise(liste_url_info):
    for url in liste_url_info :
        response = requests.get(url)
        dico_json = response.json()
        
        CodeGreffe.append(dico_json['Data']['CodeGreffe'])
        Names.append(dico_json['Data']['Denomination'])
        Sirenes.append(dico_json['Data']['Siren'])
        Adresses.append(dico_json['Data']['Adresse']['AdresseConcat'])
        ZipCodes.append(dico_json['Data']['Adresse']['CodePostal'])
        Villes.append(dico_json['Data']['Adresse']['BureauDistributeur'])
        Pays.append(dico_json['Data']['Adresse']['CodePays'])
        Secteurs.append(dico_json['Data']['CodeActivite'])
        Descriptions_FR.append(dico_json['Data']['LibelleActivite'])
                               
    return CodeGreffe, Names, Sirenes, Adresses, ZipCodes, Villes, Pays, Secteurs, Descriptions_FR

CodeGreffe, Names, Sirenes, Adresses, ZipCodes, Villes, Pays, Secteurs, Descriptions_FR = information_entreprise(liste_url_info)  
        
print(Names)
print(Adresses)
    

['SGAM AG2R LA MONDIALE', 'ASTERA', 'PRO BTP', 'CAISSE FEDERALE DE CREDIT MUTUEL', 'COOPERATIVE U ENSEIGNE', 'COOPERL ARC ATLANTIQUE', 'COVEA', 'CREDIT MUTUEL ARKEA', 'CRISTAL UNION', 'GCC', 'ETABLISSEMENTS J. SOUFFLET', 'APICIL TRANSVERSE', 'BPCE', 'GROUPE VYV', 'GIE KLESIA ADP', 'La Poste', 'SOCIETE COOPERATIVE GROUPEMENTS D ACHATS DES CENTRES LECLERC', 'MACSF FINANCEMENT', 'MAIF SOCIETE ALPHA', 'MAISADOUR', 'SOCIETE ANONYME DES GALERIES LAFAYETTE', 'MUTUELLE ASSURANCE DES COMMERCANTS ET INDUSTRIELS DE FRANCE ET DES CADRES ET SALARIES DE L INDUSTRIE ET DU COMMERCE', 'NAVAL GROUP', 'IDEMIA France', 'Orano ', 'REGIE AUTONOME DES TRANSPORTS PARISIENS', 'GROUPE ROCHER OPERATIONS', 'SNCF', 'LES MOUSQUETAIRES', 'SOCIETE COOPERATIVE AGRICOLE ARTERRIS', 'SOCIETE COOPERATIVE AGRICOLE NATUP', 'SMA SA', 'SODIAAL UNION', 'TEREOS PARTICIPATIONS', 'COOPERATIVE EUREDEN', 'SCHIEVER INTERNATIONAL', 'GROUPE ATLANTIC SYNERGY', 'GUEUDET FRERES', 'BY MY CAR GROUP', 'DECATHLON FRANCE', 'CLARINS', 'GROUPE 

**Construction des dataframes avec les tableaux résultants :**

In [50]:
data_info = pd.DataFrame({'CodeGreffe': CodeGreffe, 'Sirenes': Sirenes, 'Adresses':Adresses, 'Zipcodes': ZipCodes, 'Villes':Villes, 'Pays':Pays, 'Secteurs':Secteurs, 'Description':Descriptions_FR,},
                      index = Names)
data_info

,CodeGreffe,Sirenes,Adresses,Zipcodes,Villes,Pays,Secteurs,Description
SGAM AG2R LA MONDIALE,7501,502858418,14 BOULEVARD MALESHERBES,75008,PARIS,None,6512Z,Autres assurances
ASTERA,7608,570501551,39 RUE DES AUGUSTINS,76000,ROUEN,None,6420Z,Activités des sociétés holding
PRO BTP,7501,394164966,7 RUE DU REGARD,75006,PARIS 6,None,6530Z,Caisses de retraite
CAISSE FEDERALE DE CREDIT MUTUEL,6752,588505354,4 RUE F GUILLAUME RAIFFEISEN,67000,STRASBOURG,None,6419Z,Autres intermédiations monétaires
COOPERATIVE U ENSEIGNE,9401,304602956,BATIMENT MONTREAL PARC TERTIAIRE ICADE 20 RUE ...,94150,RUNGIS,None,4617A,Centrales d'achat alimentaires
COOPERL ARC ATLANTIQUE,2202,383986874,ZONE INDUSTRIELLE 7 RUE DE LA JEANNAIE MAROUE,22400,LAMBALLE ARMOR,None,1011Z,Transformation et conservation de la viande de...
COVEA,7501,450527916,86 RUE SAINT LAZARE,75009,PARIS,None,7010Z,Activités des sièges sociaux
CREDIT MUTUEL ARKEA,2901,775577018,1 RUE LOUIS LICHOU,29480,LE RELECQ KERHUON,None,6419Z,Autres intermédiations monétaires
CRISTAL UNION,1001,421343369,ROUTE D ARCIS SUR AUBE,10700,VILLETTE SUR AUBE,None,1081Z,Fabrication de sucre
GCC,7803,407794551,226 AVENUE DU MARECHAL FOCH,78130,LES MUREAUX,None,4120B,Construction d'autres bâtiments


**Construction des urls pour appeler l'API et récupérer les comptes entreprises (pour une année) :**

In [68]:
def url_compte_entreprise(data, annee): #attention à mettre l'année en format String
    liste_url = []
    for i in range(0,len(data)):
        url = ''
        if data['pays'][i] == 'France':
            SIRET = data['SIRET'][i]
            SIRET = re.sub(' ',  '',SIRET)
            SIRET = re.sub('\u200e',  '',SIRET) #une des lignes est problématique
            SIRET = SIRET[:-5] #traitement pour avoir le SIREN
            prefixe = 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/'
            suffixe = '&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46'
            j = prefixe + SIRET + '?millesime=' + annee + suffixe
            liste_url.append(j)
    return liste_url
        
liste_url_compte = url_compte_entreprise(data, '2019')
print("Coût en Credit Datainfo : " + str(len(liste_url_compte)*5))
display(liste_url_compte)

Coût en Credit Datainfo : 250


['https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/502858418?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/570501551?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/394164966?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/588505354?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/304602956?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfogreffe.fr/api/v1/Entreprise/ComptesAnnuels/383986874?millesime=2019&token=kuCIXNQ9mwGqxNjLHwJQSp0aXQtMzIUqU1U2KrB5UtLUbMKRAjOfChY74AZoIb46',
 'https://api.datainfo

**Appeler l'API, attention une entreprise = 5 crédits :**

In [73]:
def compte_entreprise(data, annee):
    CA_Table = []
    EBITDA_Table = []
    EBIT_Table = []
    RN_Table = []
    liste_url_compte = url_compte_entreprise(data, annee)
    for url in liste_url_compte :
        try:
            response = requests.get(url, timeout=3)  #délai de 5 à 8 min si timeout non présent       
        
            if response.status_code > 400 :
                CA_Table.append(' ')
                EBITDA_Table.append(' ')
                EBIT_Table.append(' ')
                RN_Table.append(' ')

            else :
                dico_json = response.json()

                CA = 0
                Amortissement1 = 0
                Amortissement2 = 0
                Amortissement3 = 0
                Amortissement4 = 0
                Resultat_exploi = 0
                Resultat_net = 0

                for i in range(len(dico_json['Data']['Postes'])):        
                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CFL':
                        CA = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CGA':
                        Amortissement1 = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CGB':
                        Amortissement2 = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CGC':
                        Amortissement3 = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CGD':
                        Amortissement4 = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'CGG':
                        Resultat_exploi = dico_json['Data']['Postes'][i]['Valeur']

                    if dico_json['Data']['Postes'][i]['CodeInfogreffe'] == 'DHN':
                        Resultat_net = dico_json['Data']['Postes'][i]['Valeur']

                CA_Table.append(int(CA))
                EBITDA_Table.append(int(Resultat_exploi) + int(Amortissement1) + int(Amortissement2) + int(Amortissement3) + int(Amortissement4))
                EBIT_Table.append(int(Resultat_exploi))
                RN_Table.append(int(Resultat_net))
                
        except requests.exceptions.RequestException:
            
                CA_Table.append(' ')
                EBITDA_Table.append(' ')
                EBIT_Table.append(' ')
                RN_Table.append(' ')
            
    return CA_Table, EBITDA_Table, EBIT_Table, RN_Table

annee = '2019'            
CA2019_Table, EBITDA2019_Table, EBIT2019_Table, RN2019_Table = compte_entreprise(data, annee)

**Construction des Dataframes :**

In [74]:
data_compte_2019 = pd.DataFrame({'CA2019_Table': CA2019_Table, 'EBITDA2019_Table': EBITDA2019_Table, 'EBIT2019_Table':EBIT2019_Table, 'RN2019_Table':RN2019_Table, },
                      index = Names)
data_compte_2019

,CA2019_Table,EBITDA2019_Table,EBIT2019_Table,RN2019_Table
SGAM AG2R LA MONDIALE,,,,
ASTERA,,,,
PRO BTP,,,,
CAISSE FEDERALE DE CREDIT MUTUEL,,,,
COOPERATIVE U ENSEIGNE,,,,
COOPERL ARC ATLANTIQUE,,,,
COVEA,,,,
CREDIT MUTUEL ARKEA,,,,
CRISTAL UNION,960222140,-23422720,-76940929,-97606914
GCC,533793742,30978006,3160116,8212508


**Partie à ignorer : test divers :**

In [85]:
print(CA2018_Table)
print(EBITDA2018_Table)
print(EBIT2018_Table)
print(RN2018_Table)

['', 4969573000, 1003706123, 498580565, '']
['', 90616000, 110006716, 22403644, '']
['', 47065000, 68013405, 1991089, '']
['', 0, 34584502, 955041, '']


Remarque : Seul le premier fait partie des comptes manquants. Pour le dernier Groupe Soufflet, on aurait dû avoir le résultat.

In [86]:
CA2019_Table, EBITDA2019_Table, EBIT2019_Table, RN2019_Table = compte_entreprise(data, '2019')

In [87]:
print(CA2019_Table)
print(EBITDA2019_Table)
print(EBIT2019_Table)
print(RN2019_Table)

['', '', 960222140, 533793742, '']
['', '', -23422720, 30978006, '']
['', '', -76940929, 3160116, '']
['', '', -97606914, 8212508, '']


In [88]:
CA2017_Table, EBITDA2017_Table, EBIT2017_Table, RN2017_Table = compte_entreprise(data, '2017')

In [89]:
print(CA2017_Table)
print(EBITDA2017_Table)
print(EBIT2017_Table)
print(RN2017_Table)

['', 0, 1354878215, 428318702, '']
['', -448000, 216281585, 28740052, '']
['', -448000, 156175368, 5872851, '']
['', 12004000, 232855506, 6186961, '']


Convertion
1. CFL => Chiffre d'affaire
2. CGA + CGB + CGC + CGD => Dotation aux amortissements
3. CGG => Résultat d'exploitation = EBIT
4. CGG + CGA + CGB + CGC + CGD => EBITDA
5. DHN => Résultat Net


**Construction du fichier excel :**

In [75]:
data_fusion = pd.concat([data_info,data_compte_2019],axis=1)

In [76]:
def dataframes_to_excel(df):
    # create excel writer object
    writer = pd.ExcelWriter(r'C:\Users\Andy\Documents\3A\PROJET ARX\resultats.xlsx',engine='openpyxl')
    # write dataframe to excel
    df.to_excel(writer)
    # save the excel
    writer.save()
    return print('DataFrame is written successfully to Excel File.')

dataframes_to_excel(data_fusion)

DataFrame is written successfully to Excel File.


# FIN

**Partie brouillon :**

In [19]:
A = [2,4,6,8,10,12]
B = [1,3,5,7,9,11]
N = ['azer','ert','rty','tyu','yui','iop']
test = pd.DataFrame({'A': A, 'B' : B}, index = N)
test

,A,B
azer,2,1
ert,4,3
rty,6,5
tyu,8,7
yui,10,9
iop,12,11


In [21]:
C = [14,47,25,58,36,69]
Ctest = pd.DataFrame({'C': C}, index = N)
test2 = pd.concat([test,Ctest],axis=1)
test2

,A,B,C
azer,2,1,14
ert,4,3,47
rty,6,5,25
tyu,8,7,58
yui,10,9,36
iop,12,11,69
